# Tuning and deploy a foundation model


**Learning Objective**

1. Learn how to generate a JSONL file for PaLM tuning
1. Learn how to launch a tuning job on Vertex Pipeline
1. Learn how to deploy and query a tuned LLM
1. Learn how to evaluate a tuned LLM


Creating an LLM requires massive amounts of data, significant computing resources, and specialized skills. In this notebook, you'll learn how tuning allows you to customize a PaLM foundation model on Vertex Generative AI studio for more specific tasks or knowledge domains.
While the prompt design is excellent for quick experimentation, if training data is available, you can achieve higher quality by tuning the model. Tuning a model enables you to customize the model response based on examples of the task you want the model to perform.

For more details on tuning have a look at the [official documentation](https://cloud.google.com/vertex-ai/docs/generative-ai/models/tune-models).

**Quota**: Tuning the `text-bison@001`  model uses the `tpu-v3-8` training resources and the accompanying quotas from your Google Cloud project. Each project has a default quota of eight v3-8 cores, which allows for one to two concurrent tuning jobs. If you want to run more concurrent jobs you need to request additional quota via the [Quotas page](https://console.cloud.google.com/iam-admin/quotas).

**Costs:** This tutorial uses a billable component of Google Cloud `Vertex AI Generative AI Studio`.
Learn about [Vertex AI pricing](https://cloud.google.com/vertex-ai/pricing),
and use the [Pricing Calculator](https://cloud.google.com/products/calculator/)
to generate a cost estimate based on your projected usage.


## Setup

In [ ]:
import os
import warnings

warnings.filterwarnings("ignore")
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"

In [2]:
import time

import evaluate
import pandas as pd
from google.cloud import aiplatform, bigquery
from sklearn.model_selection import train_test_split
from vertexai.preview.language_models import TextGenerationModel

In [3]:
REGION = "us-central1"
PROJECT_ID = !(gcloud config get-value project)
PROJECT_ID = PROJECT_ID[0]
BUCKET_NAME = PROJECT_ID
BUCKET_URI = f"gs://{BUCKET_NAME}"

In [ ]:
!gsutil ls $BUCKET_URI || gsutil mb -l $REGION -p $PROJECT_ID $BUCKET_URI

## Training Data


In this notebook, we will be tuning the Vertex PaLM vertex using the Python SDK on a questions & answers dataset  from StackOverflow. 
Our first step will be to query the StackOverflow data on BigQuery Public Datasets, limiting to questions with the `python` tag, and `accepted` answers from 2020-01-01 only. 

We will limit the dataset to 1000 samples, 800 of which will be used to tune the LLM and the rest for evaluating the tuned model.
The second step will be to convert the dataset into a JSONL format, with one example per line, so that the tuning job can consume it.


Next let us run the query to assemble our dataset into the DataFrame `df`:

In [4]:
%%bigquery df

SELECT CONCAT(q.title, q.body) as input_text, a.body AS output_text
FROM
    `bigquery-public-data.stackoverflow.posts_questions` q
JOIN
    `bigquery-public-data.stackoverflow.posts_answers` a
ON
    q.accepted_answer_id = a.id
WHERE
    q.accepted_answer_id IS NOT NULL AND
    REGEXP_CONTAINS(q.tags, "python") AND
    a.creation_date >= "2020-01-01"
LIMIT
    1000

In [5]:
df.head()

The column `input_text` corresponds to the actual questions asked by the StackOverflow users, while the `output_text` column corresponds to the correct answers. From this dataset of 1000 questions-answers pairs, we will now need to generate a JSONL file with one example per line in the format:

```python
{'input_text': <QUESTION>, 'output_text': <ANSWER>}
```

This is the format we need to tune our LLM model.

Let's first split the data into training and evaluation. To tune PaLM for a Q&A task we advise 100+ training examples. In this case you will use 800.


In [6]:
# split is set to 80/20
train, evaluation = train_test_split(df, test_size=0.2)
print(len(train))

For tuning, the training data first needs to be converted into a JSONL format, which is very easy in Pandas:

In [ ]:
training_data_filename = "tune_data_stack_overflow_python_qa.jsonl"

train.to_json(training_data_filename, orient="records", lines=True)

Let's inspect the first line of the JSONL file we just created:

In [ ]:
!head -n 1 $training_data_filename

You can then export the local file to GCS, so that it can be used by Vertex AI for the tuning job.

In [ ]:
!gsutil cp $training_data_filename $BUCKET_URI

You can check to make sure that the file successfully transferred to your Google Cloud Storage bucket:

In [ ]:
TRAINING_DATA_URI = f"{BUCKET_URI}/{training_data_filename}"

!gsutil ls -al $TRAINING_DATA_URI

### Model Tuning
Now it's time to start to tune a model. You will use the Vertex AI SDK to submit our tuning job.

#### Recommended Tuning Configurations
✅ Here are some recommended configurations for tuning a foundation model based on the task, in this example Q&A. You can find more in the [documentation](https://cloud.google.com/vertex-ai/docs/generative-ai/models/tune-models).

Question Answering task:
- Make sure that your train dataset size is 100+
- Choose your training steps in the range 100-500. You can try more than one value to get the best performance on a particular dataset (e.g. 100, 200, 500)

In [ ]:
aiplatform.init(project=PROJECT_ID, location=REGION)

model = TextGenerationModel.from_pretrained("text-bison@001")

Next it's time to start your tuning job. 

**Disclaimer:** Tuning and deploying a LLM model takes time. 
For 100 train steps, it takes around 1h. For 500 train steps, it takes around 3h30.

In [ ]:
TRAIN_STEPS = 100
MODEL_NAME = f"asl-palm-text-tuned-model-{time.time()}"

model.tune_model(
    training_data=TRAINING_DATA_URI,
    model_display_name=MODEL_NAME,
    train_steps=TRAIN_STEPS,
    # Tuning can only happen in the "europe-west4" location for now
    tuning_job_location="europe-west4",
    # Model can only be deployed in the "us-central1" location for now
    tuned_model_location="us-central1",
)

print("Model name:", MODEL_NAME)

## Retrieve the tuned model from your Vertex AI Model registry


When your tuning job is finished, your model will be available on Vertex AI Model Registry. The next cell shows you how to list tuned models.

In [7]:
model = TextGenerationModel.from_pretrained("text-bison@001")
model.list_tuned_model_names()

You can also use the Google Cloud Console UI to view all of your models in [Vertex AI Model Registry](https://console.cloud.google.com/vertex-ai/models?). 

It's time to get predictions. First you need to get the latest tuned model from the Vertex AI Model registry.

In [8]:
tuned_model = TextGenerationModel.get_tuned_model(
    model.list_tuned_model_names()[-1]
)

Now you can start sending a prompt to the API. Feel free to update the following prompt:

In [9]:
PROMPT = """
How can I store my TensorFlow checkpoint on Google Cloud Storage?

Python example:

"""

print(tuned_model.predict(PROMPT))

## Evaluation


It's essential to evaluate your model to understand its performance. Evaluation can be done in an automated way using evaluation metrics like F1, Bleu, or Rouge. You can also leverage human evaluation methods. Human evaluation methods involve asking humans to rate the quality of the LLM's answers. This can be done through crowdsourcing or by having experts evaluate the responses. Some standard human evaluation metrics include fluency, coherence, relevance, and informativeness. Often you want to choose a mix of evaluation metrics to get a good understanding of your model performance. 


Among other metrics we will compute the following two metrics that provide crude measures albeit automated of how two texts may have the same meaning: 
- The [BLEU](https://en.wikipedia.org/wiki/BLEU) evaluation metric is a sort of **precision** metric, measuring the proportion of $n$-grams in the generated sentence matching $n$-grams in the reference sentence. It goes from 0 to 1 with a higher score for more similar sentences. BLEU1 considers uni-grams only, while BLEU2 considers bi-grams. 

- The [ROUGE](https://en.wikipedia.org/wiki/ROUGE_(metric)) evaluation metric is a sort of **recall** metric, measuring the proportion of $n$-grams in the reference sentence that are matched by $n$-grams in the generated sentence. It goes from 0 to 1 with a higher score for more similar sentences. ROUGE1 considers uni-grams only, while ROUGE2 considers bi-grams.


We will use  [evaluate](https://github.com/huggingface/evaluate/tree/main) to to compute the scores.
Earlier in the notebook, you created a train and eval dataset. Now it's time to take some of the eval data. You will use the questions to get a response from our tuned model, and the answers we will use as a reference:
- **Candidates**: Answers generated by the tuned model.
- **References**: Original answers that we will use to compare



Let us first select a sample of our evaluation set:

In [10]:
# you can change the number of rows you want to use
EVAL_ROWS = 60
INPUT_LIMIT = 10000  # characters
evaluation = evaluation[evaluation.input_text.apply(len) <= INPUT_LIMIT]
evaluation = evaluation.head(EVAL_ROWS)
evaluation.head()

The function in the cell below will query our tuned model using the `evaluation.input_text` and store the ground truth in `evaluation.output_text` in a DataFrame next to the model answers:

In [11]:
def create_eval_data(model, evaluation):
    model_answers = []

    for prompt in evaluation.input_text:
        response = model.predict(prompt)
        model_answers.append(response.text)

    eval_df = pd.DataFrame(
        {"candidate": model_answers, "reference": evaluation.output_text}
    )
    mask = eval_df.candidate == ""
    return eval_df[~mask]

In [12]:
eval_df = create_eval_data(model, evaluation)

In [13]:
eval_df.head()

The function in the next cell computes the uni-gram BLEU and ROUGE scores. It averages these scores over all the reference answers and those generated by our tuned model, giving scores that can serve as performance metrics for our model.

In [14]:
def compute_scores(eval_data):
    predictions = eval_data.candidate.tolist()
    references = eval_data.reference.tolist()
    rouge = evaluate.load("rouge")
    bleu = evaluate.load("bleu")
    rouge_value = rouge.compute(predictions=predictions, references=references)[
        "rouge1"
    ]
    bleu_value = bleu.compute(predictions=predictions, references=references)[
        "bleu"
    ]
    return {"rouge": rouge_value, "bleu": bleu_value}

In [15]:
compute_scores(eval_df)

Given two versions of the model (possibly tuned with a different amount of data or training steps), you can now compare the scores to decide which one is the best. However, remember that these automated metrics are very crude proxy of human assessment. 

## Acknowledgement 

This notebook is adapted from a [tutorial](https://github.com/GoogleCloudPlatform/generative-ai/blob/main/language/tuning/getting_started_tuning.ipynb)
written by Polong Lin.

Copyright 2023 Google LLC

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at

     https://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.